In [9]:
import pandas as pd
import numpy as np
from scipy.stats import chisquare
import re
    

In [10]:
# Load sample dataset
population_filename = "../data/population-since-10000bc.csv"
population_data = pd.read_csv(population_filename)
popaulation_column = 'Population (historical)' # column with numerical data

population_data.head()

,Entity,Code,Year,Population (historical)
0,Afghanistan,AFG,-10000,14737
1,Afghanistan,AFG,-9000,20405
2,Afghanistan,AFG,-8000,28253
3,Afghanistan,AFG,-7000,39120
4,Afghanistan,AFG,-6000,54166


In [148]:
MIN_ORDERS_OF_MAGNITUDE = 2

def check_magnitude_range(series):
    p1 = series.abs().quantile(0.01)
    p99 = series.abs().quantile(0.99)
    
    
    magnitude_range = np.log10(p99) - np.log10(p1)
    
    return magnitude_range

print(check_magnitude_range(population_data[popaulation_column]))

6.482972250046383


In [149]:
benford_probs = {
    d: np.log10(1 + 1/d) for d in range(1, 10)
}

benford_first_digits = list(benford_probs.values())
benford_last_digits = [0.1] * 10

def test_goodness_of_fit_chisquare(observed_counts, expected, label='first'):
    print(f"\nTesting goodness of fit for {label} digits:")

    # Example observed data: digit counts from your dataset
    observed_counts = np.array(observed_counts)  # replace with your counts
    n = observed_counts.sum()

    # Expected counts under Benford's law
    expected_counts = np.array([p * n for p in expected])
    print(f"observed counts for {label} digits:", observed_counts)
    print(f"expected counts for {label} digits:", expected_counts)

    # Chi-square goodness of fit test
    chi2_stat, p_value = chisquare(f_obs=observed_counts, f_exp=expected_counts)

    print(f"\nChi-square goodness of fit test for {label} digits:")
    print("Chi-square statistic:", chi2_stat)
    print("p-value:", p_value)

    if p_value > 0.05:
        print(f"Fail to reject null: data follows Benford's law (at 5% significance) for {label} digits.")
    else:
        print(f"Reject null: data does not follow Benford's law for {label} digits.")
        
    return chi2_stat, p_value

def test_goodness_of_fit_mad(observed_counts, expected, label='first'):
    print(f"\nTesting goodness of fit (MAD) for {label} digits:")
    observed_counts = np.array(observed_counts)

    # Convert observed counts to proportions
    observed_probs = observed_counts / observed_counts.sum()
    print(f"observed proportions for {label} digits:", observed_probs)
    print(f"expected proportions for {label} digits:", expected)

    # Mean Absolute Deviation
    mad = np.mean(np.abs(observed_probs - expected))
    
    category = ''

    # Classify conformity (per Nigrini's thresholds)
    if mad < 0.006:
        print(f"{label.capitalize()} digits show Close conformity with Benford's Law")
        category = 'Close conformity'
    elif mad < 0.012:
        print(f"{label.capitalize()} digits show Acceptable conformity with Benford's Law")
        category = 'Acceptable conformity'
    else:
        print(f"{label.capitalize()} digits show Non-conformity with Benford's Law")
        category = 'Non-conformity'
        
    return mad, category


In [150]:
def get_dataset_stats(dataset , print_output=True):
    
    df = pd.read_csv(f'../data/{dataset}')
    
    numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    print(f'Found {len(numerical_cols)} numerical columns in {dataset}: {numerical_cols} \n\n')
    
    result_list = []
    
    print('COLUMN-WISE ANALYSIS:')
    for col in numerical_cols:
        print('----------------------------------\n')
        
        
        col_data = df[col].abs()
        col_data = col_data.replace(0, np.nan).dropna()
        
        magnitude_range = np.round(check_magnitude_range(col_data), 2)
        
        print(f'Column "{col}" has a magnitude range of {magnitude_range}.')

        if magnitude_range == MIN_ORDERS_OF_MAGNITUDE:
            print(f'Since magnitude range({magnitude_range} = 2), the data should start to show Benford-like properties.')
            
        elif (magnitude_range > MIN_ORDERS_OF_MAGNITUDE):
            print(f'Since magnitude range({magnitude_range}) > 2, it is a good candidate for Benford analysis.')

        else:
            print(f'Since magnitude range({magnitude_range}) < 2, "{col}" is unlikely to follow Benford\'s Law.')
            
        benfords_ten_digits = [0] + benford_first_digits

        # Extract first and last digits
        first_digits = col_data.apply(extract_first_digit).value_counts().sort_index()
        # Ensure digits 1-9 are present (missing digits get count 0)
        first_digits = first_digits.reindex(range(1, 10), fill_value=0)
        print('first_digits:', first_digits)
        first_digits_ratio = first_digits / first_digits.sum()

        last_digits = col_data.apply(extract_last_digit).value_counts().sort_index()
        last_digits = last_digits.reindex(range(0, 10), fill_value=0)
        print('last_digits:', last_digits)
        last_digits_ratio = last_digits / last_digits.sum()
        
        benford_first_digit_series = pd.Series(benford_first_digits)
        benford_first_digit_series.index = benford_first_digit_series.index
        
        
        results = pd.DataFrame({
            'First Digit Ratio': first_digits_ratio,
            'Expected Benford First Digit Ratio': benford_first_digit_series,
            'Last Digit Ratio': last_digits_ratio,
        })


        if print_output:
            display(f'Distribution of first and last digits for column "{col}":')
            display(results.fillna(0).round(4))
        
        # chisquare_first, pvalue_first = test_goodness_of_fit_chisquare(first_digits.values.tolist(), benford_first_digits, label='first')
        # chisquare_last, pvalue_last = test_goodness_of_fit_chisquare(last_digits.values.tolist(), benford_last_digits, label='last')
        
        mad_first, mad_first_category = test_goodness_of_fit_mad(first_digits.values.tolist(), benford_first_digits, label='first')
        mad_last, mad_last_category = test_goodness_of_fit_mad(last_digits.values.tolist(), benford_last_digits, label='last')
        
        final_result = {
            'dataset': dataset,
            'column': col,
            'magnitude_range': magnitude_range,
            'mad_first': mad_first,
            'mad_first_category': mad_first_category,
            'mad_last': mad_last,
            'mad_last_category': mad_last_category,
        }
        
        first_digits_ratio_cols = {f"first_digit_{i}": v for i, v in first_digits_ratio.items()}
        last_digits_ratio_cols = {f"last_digit_{i}": v for i, v in last_digits_ratio.items()}
        final_result.update(first_digits_ratio_cols)
        final_result.update(last_digits_ratio_cols)
        result_list.append(final_result)
        
    return result_list


def extract_first_digit(x):
    s = str(abs(x))  # remove sign
    s = re.sub(r"[^0-9]", "", s)  # keep only digits
    for ch in s:
        if ch != "0":
            return int(ch)
    return None


def extract_last_digit(x):
    """Extract last digit from a number, ignoring trailing .0 if present."""
    try:
        s = str(x).strip()
        
        # If the number ends with ".0", drop it
        if s.endswith(".0"):
            s = s[:-2]
        
        s = s.replace(".", "").replace("-", "")  # remove dot and minus
        return int(s[-1]) if s else None
    
    except Exception:
        return None
    
temp = get_dataset_stats(population_filename, print_output=False)

Found 2 numerical columns in ../data/population-since-10000bc.csv: ['Year', 'Population (historical)'] 


COLUMN-WISE ANALYSIS:
----------------------------------

Column "Year" has a magnitude range of 1.37.
Since magnitude range(1.37) < 2, "Year" is unlikely to follow Benford's Law.
first_digits: Year
1    49402
2     6576
3      387
4      387
5      388
6      388
7      389
8      388
9      388
Name: count, dtype: int64
last_digits: Year
0    12968
1     5278
2     5249
3     5257
4     4988
5     4992
6     4997
7     4988
8     4988
9     4988
Name: count, dtype: int64

Testing goodness of fit (MAD) for first digits:
observed proportions for first digits: [0.84170174 0.11204062 0.00659363 0.00659363 0.00661067 0.00661067
 0.00662771 0.00661067 0.00661067]
expected proportions for first digits: [0.3010299956639812, 0.17609125905568124, 0.12493873660829993, 0.09691001300805642, 0.07918124604762482, 0.06694678963061322, 0.05799194697768673, 0.05115252244738129, 0.04575749056067514

In [ ]:
# get_dataset_stats('population-since-10000bc.csv')

Found 2 numerical columns in population-since-10000bc.csv: ['Year', 'Population (historical)'] 


COLUMN-WISE ANALYSIS:
----------------------------------

Column "Year" has a magnitude range of 2.0.
Since magnitude range(2.0 = 2), the data should start to show Benford-like properties.


'Distribution of first and last digits for column "Year":'

,First Digit Ratio,Expected Benford First Digit Ratio,Last Digit Ratio
0,0.0000,0.3010,0.2209
1,0.8417,0.1761,0.0899
2,0.1120,0.1249,0.0894
3,0.0066,0.0969,0.0896
4,0.0066,0.0792,0.0850
5,0.0066,0.0669,0.0851
6,0.0066,0.0580,0.0851
7,0.0066,0.0512,0.0850
8,0.0066,0.0458,0.0850
9,0.0066,0.0000,0.0850


First digits - MAD: 0.12014927566412029
First digits show Non-conformity with Benford's Law
Last digits - MAD: 0.024189255958972967
Last digits show Non-conformity with Benford's Law
   First Digit Ratio  Expected Benford First Digit Ratio  Last Digit Ratio
0                NaN                            0.301030          0.220946
1           0.841702                            0.176091          0.089926
2           0.112041                            0.124939          0.089431
3           0.006594                            0.096910          0.089568
4           0.006594                            0.079181          0.084985
5           0.006611                            0.066947          0.085053
6           0.006611                            0.057992          0.085138
7           0.006628                            0.051153          0.084985
8           0.006611                            0.045757          0.084985
9           0.006611                                 NaN          0

'Distribution of first and last digits for column "Population (historical)":'

,First Digit Ratio,Expected Benford First Digit Ratio,Last Digit Ratio
0,0.0000,0.3010,0.1386
1,0.2836,0.1761,0.0949
2,0.1863,0.1249,0.0946
3,0.1331,0.0969,0.0973
4,0.1020,0.0792,0.0975
5,0.0809,0.0669,0.0942
6,0.0654,0.0580,0.0961
7,0.0549,0.0512,0.0939
8,0.0488,0.0458,0.0965
9,0.0451,0.0000,0.0966


First digits - MAD: 0.005578495913507595
First digits show Close conformity with Benford's Law
Last digits - MAD: 0.007719476991000174
Last digits show Acceptable conformity with Benford's Law
   First Digit Ratio  Expected Benford First Digit Ratio  Last Digit Ratio
0                NaN                            0.301030          0.138597
1           0.283580                            0.176091          0.094855
2           0.186296                            0.124939          0.094583
3           0.133062                            0.096910          0.097300
4           0.101953                            0.079181          0.097521
5           0.080914                            0.066947          0.094176
6           0.065427                            0.057992          0.096060
7           0.054933                            0.051153          0.093870
8           0.048769                            0.045757          0.096485
9           0.045067                                 NaN 

In [151]:
datasets = pd.read_csv('../data/exploration-datasets/datasets_overview.csv')
display(datasets.head())

def test_runner(datasets=datasets):
    overview_stats = []
    
    for idx, row in datasets.iterrows():
        dataset_name = row['filename']
        print(f'Processing dataset: {dataset_name}')
        dataset_stats = get_dataset_stats("exploration-datasets/"+dataset_name)
        overview_stats.extend(dataset_stats)
        
    return overview_stats

,filename,title,description,source,num_rows,url,date_downloaded
0,share-of-population-in-extreme-poverty.csv,Share of population living in extreme poverty,Percentage of population living in households ...,World Bank Poverty and Inequality Platform (2025),2743,https://ourworldindata.org/grapher/share-of-po...,2025-09-21
1,gdp-per-capita-maddison-project-database.csv,GDP per capita,Average economic output per person in a countr...,Bolt and van Zanden – Maddison Project Databas...,21586,https://ourworldindata.org/grapher/gdp-per-cap...,2025-09-21
2,distribution-of-population-poverty-thresholds.csv,Distribution of population between different p...,Number of people living in households with an ...,World Bank Poverty and Inequality Platform (2025),2743,https://ourworldindata.org/grapher/distributio...,2025-09-21
3,human-development-index.csv,Human Development Index,The Human Development Index (HDI) is a summary...,"UNDP, Human Development Report (2025)",6683,https://ourworldindata.org/grapher/human-devel...,2025-09-21
4,prevalence-of-undernourishment.csv,Share of people who are undernourished,Share of the population whose daily food intak...,Food and Agriculture Organization of the Unite...,4683,https://ourworldindata.org/grapher/prevalence-...,2025-09-21


In [152]:
stats = test_runner(datasets)
stats_df = pd.DataFrame(stats)
stats_df.to_csv('../data/outputs/columns_overview.csv', index=False)

Processing dataset: share-of-population-in-extreme-poverty.csv
Found 2 numerical columns in exploration-datasets/share-of-population-in-extreme-poverty.csv: ['Year', 'Share of population in poverty ($3 a day, 2021 prices) - Income or consumption consolidated'] 


COLUMN-WISE ANALYSIS:
----------------------------------

Column "Year" has a magnitude range of 0.01.
Since magnitude range(0.01) < 2, "Year" is unlikely to follow Benford's Law.
first_digits: Year
1     675
2    2068
3       0
4       0
5       0
6       0
7       0
8       0
9       0
Name: count, dtype: int64
last_digits: Year
0    291
1    293
2    306
3    256
4    253
5    281
6    262
7    257
8    273
9    271
Name: count, dtype: int64


'Distribution of first and last digits for column "Year":'

,First Digit Ratio,Expected Benford First Digit Ratio,Last Digit Ratio
0,0.0000,0.3010,0.1061
1,0.2461,0.1761,0.1068
2,0.7539,0.1249,0.1116
3,0.0000,0.0969,0.0933
4,0.0000,0.0792,0.0922
5,0.0000,0.0669,0.1024
6,0.0000,0.0580,0.0955
7,0.0000,0.0512,0.0937
8,0.0000,0.0458,0.0995
9,0.0000,0.0000,0.0988



Testing goodness of fit (MAD) for first digits:
observed proportions for first digits: [0.24608093 0.75391907 0.         0.         0.         0.
 0.         0.         0.        ]
expected proportions for first digits: [0.3010299956639812, 0.17609125905568124, 0.12493873660829993, 0.09691001300805642, 0.07918124604762482, 0.06694678963061322, 0.05799194697768673, 0.05115252244738129, 0.04575749056067514]
First digits show Non-conformity with Benford's Law

Testing goodness of fit (MAD) for last digits:
observed proportions for last digits: [0.10608822 0.10681735 0.11155669 0.09332847 0.09223478 0.10244258
 0.09551586 0.09369304 0.09952607 0.09879694]
expected proportions for last digits: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
Last digits show Close conformity with Benford's Law
----------------------------------

Column "Share of population in poverty ($3 a day, 2021 prices) - Income or consumption consolidated" has a magnitude range of 3.6.
Since magnitude range(3.6) > 2

'Distribution of first and last digits for column "Share of population in poverty ($3 a day, 2021 prices) - Income or consumption consolidated":'

...

,First Digit Ratio,Expected Benford First Digit Ratio,Last Digit Ratio
0,0.0,0.3010,0.0909
1,0.0,0.1761,0.1364
2,1.0,0.1249,0.1364
3,0.0,0.0969,0.0909
4,0.0,0.0792,0.0909
5,0.0,0.0669,0.0909
6,0.0,0.0580,0.0909
7,0.0,0.0512,0.0909
8,0.0,0.0458,0.0909
9,0.0,0.0000,0.0909



Testing goodness of fit (MAD) for first digits:
observed proportions for first digits: [0. 1. 0. 0. 0. 0. 0. 0. 0.]
expected proportions for first digits: [0.3010299956639812, 0.17609125905568124, 0.12493873660829993, 0.09691001300805642, 0.07918124604762482, 0.06694678963061322, 0.05799194697768673, 0.05115252244738129, 0.04575749056067514]
First digits show Non-conformity with Benford's Law

Testing goodness of fit (MAD) for last digits:
observed proportions for last digits: [0.09090909 0.13636364 0.13636364 0.09090909 0.09090909 0.09090909
 0.09090909 0.09090909 0.09090909 0.09090909]
expected proportions for last digits: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
Last digits show Non-conformity with Benford's Law
----------------------------------

Column "Cost of sequencing a full human genome" has a magnitude range of 5.28.
Since magnitude range(5.28) > 2, it is a good candidate for Benford analysis.
first_digits: Cost of sequencing a full human genome
1    7
2    1
3   

'Distribution of first and last digits for column "Cost of sequencing a full human genome":'

,First Digit Ratio,Expected Benford First Digit Ratio,Last Digit Ratio
0,0.0000,0.3010,0.0909
1,0.3182,0.1761,0.0909
2,0.0455,0.1249,0.1364
3,0.0455,0.0969,0.0909
4,0.1364,0.0792,0.1364
5,0.1818,0.0669,0.0909
6,0.0909,0.0580,0.1364
7,0.1364,0.0512,0.0455
8,0.0000,0.0458,0.1364
9,0.0455,0.0000,0.0455



Testing goodness of fit (MAD) for first digits:
observed proportions for first digits: [0.31818182 0.04545455 0.04545455 0.13636364 0.18181818 0.09090909
 0.13636364 0.         0.04545455]
expected proportions for first digits: [0.3010299956639812, 0.17609125905568124, 0.12493873660829993, 0.09691001300805642, 0.07918124604762482, 0.06694678963061322, 0.05799194697768673, 0.05115252244738129, 0.04575749056067514]
First digits show Non-conformity with Benford's Law

Testing goodness of fit (MAD) for last digits:
observed proportions for last digits: [0.09090909 0.09090909 0.13636364 0.09090909 0.13636364 0.09090909
 0.13636364 0.04545455 0.13636364 0.04545455]
expected proportions for last digits: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
Last digits show Non-conformity with Benford's Law


In [160]:
%pip install matplotlib seaborn


  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/31/3c/80816f027b3a4a28cd2a0a6ef7f89a2db22310e945cd886ec25bfb399221/matplotlib-3.10.7-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for seaborn from https://files.pythonhosted.org/packages/83/11/00d3c3dfc25ad54e731d91449895a79e4bf2384dc3ac01809010ba88f6d5/seaborn-0.13.2-py3-none-any.whl.metadata
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/19/e8/6026ed58a64563186a9ee3f29f41261fd1828f527dd93d33b60feca63352/contourpy-1.3.3-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for cycler>=0.10 from https://files.pythonhosted.org/packages/e7/05/c19819d5e3d95294a6f5947fb9b9629efb316b96de511b418c53d245aae6/cycler-0.12.1-py3-none-any.whl.metadata
  Obtaining dependency information for fonttools>=4.22.0 from https://files.pythonhosted.org/packages/88/8f/a55b5550cd33cd1028601df41acd057d4be20efa5c958f417b0c0


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_numeric_distribution(filename, column, bins=50, figsize=(10, 6), log_y=False, kde=False):
    """
    Read CSV by filename and display a distribution plot for `column` using seaborn.
    - filename: path or filename (will try given path, else ../data/{filename}).
    - column: column name to plot.
    - bins: number of bins for histogram when many unique values.
    - figsize: matplotlib figure size.
    - log_y: use log scale on y axis when True.
    - kde: if True and histogram is used, overlay a KDE.
    """
    # load dataframe
    try:
        df = pd.read_csv(filename)
    except Exception:
        df = pd.read_csv(f'../data/{filename}')

    if column not in df.columns:
        raise ValueError(f'Column "{column}" not found in dataset')

    # coerce to numeric and drop NaNs
    series = pd.to_numeric(df[column], errors='coerce').dropna()
    if series.empty:
        raise ValueError(f'Column "{column}" contains no numeric data after coercion')

    plt.figure(figsize=figsize)

    if series.nunique() <= 30:
        counts = series.value_counts().sort_index()
        # seaborn likes DataFrame/arrays; use barplot for consistent styling
        sns.barplot(x=counts.index.astype(str), y=counts.values, color='C0', edgecolor='k')
        plt.xlabel(column)
        plt.ylabel('Count')
        plt.xticks(rotation=45, ha='right')
    else:
        # histogram with optional KDE overlay
        sns.histplot(series, bins=bins, color='C0', edgecolor='k', kde=kde)
        plt.xlabel(column)
        plt.ylabel('Frequency')

    if log_y:
        plt.yscale('log')

    plt.title(f'Distribution of "{column}"')
    plt.tight_layout()
    plt.show()


ModuleNotFoundError: No module named 'matplotlib'

In [158]:
stats_df[stats_df['magnitude_range'] > 3]

,dataset,column,magnitude_range,mad_first,mad_first_category,mad_last,mad_last_category,first_digit_1,first_digit_2,first_digit_3,...,last_digit_0,last_digit_1,last_digit_2,last_digit_3,last_digit_4,last_digit_5,last_digit_6,last_digit_7,last_digit_8,last_digit_9
1,exploration-datasets/share-of-population-in-ex...,"Share of population in poverty ($3 a day, 2021...",3.60,0.016376,Non-conformity,0.030598,Non-conformity,0.245465,0.175608,0.107294,...,0.000000,0.074489,0.098418,0.123118,0.125820,0.149363,0.121575,0.132767,0.100347,0.074103
5,exploration-datasets/distribution-of-populatio...,Number of people not in poverty (above $10 a d...,4.75,0.015554,Non-conformity,0.003425,Close conformity,0.263081,0.168313,0.138310,...,0.101720,0.099158,0.098061,0.095499,0.098427,0.093304,0.098427,0.104647,0.100988,0.109769
6,exploration-datasets/distribution-of-populatio...,Number of people in poverty (between $8.30 and...,5.92,0.006225,Acceptable conformity,0.005467,Close conformity,0.300971,0.181852,0.127334,...,0.096714,0.104929,0.103809,0.108663,0.085885,0.100822,0.104929,0.104182,0.095967,0.094100
7,exploration-datasets/distribution-of-populatio...,Number of people in poverty (between $4.20 and...,6.34,0.010343,Acceptable conformity,0.003290,Close conformity,0.343785,0.166359,0.116931,...,0.098857,0.097381,0.095537,0.097381,0.103652,0.094430,0.100332,0.101439,0.099963,0.111029
8,exploration-datasets/distribution-of-populatio...,Number of people in poverty (between $3 and $4...,6.63,0.008863,Acceptable conformity,0.004037,Close conformity,0.286849,0.161303,0.114025,...,0.091776,0.106079,0.101708,0.102900,0.104092,0.098927,0.102900,0.096146,0.092968,0.102503
9,exploration-datasets/distribution-of-populatio...,"Number of people in poverty ($3 a day, 2021 pr...",6.73,0.008245,Acceptable conformity,0.004886,Close conformity,0.309533,0.151293,0.113856,...,0.104593,0.106909,0.103435,0.102663,0.101891,0.094172,0.089927,0.091470,0.103049,0.101891
22,exploration-datasets/population-with-un-projec...,Population - Sex: all - Age: all - Variant: es...,6.21,0.008735,Acceptable conformity,0.002274,Close conformity,0.270904,0.167177,0.126900,...,0.104255,0.098554,0.097656,0.103885,0.099609,0.095386,0.099240,0.098184,0.101035,0.102196
23,exploration-datasets/population-with-un-projec...,Population - Sex: all - Age: all - Variant: me...,6.45,0.015848,Non-conformity,0.002179,Close conformity,0.278105,0.131037,0.121601,...,0.104403,0.097403,0.100802,0.100751,0.101613,0.098366,0.100802,0.096540,0.102526,0.096794
25,exploration-datasets/population.csv,Population (historical),6.48,0.005578,Close conformity,0.007719,Acceptable conformity,0.283580,0.186296,0.133062,...,0.138597,0.094855,0.094583,0.097300,0.097521,0.094176,0.096060,0.093870,0.096485,0.096553
32,exploration-datasets/co-emissions-per-capita.csv,Annual COâ emissions (per capita),5.00,0.007335,Acceptable conformity,0.031571,Non-conformity,0.288861,0.158151,0.122039,...,0.000000,0.069632,0.099861,0.123007,0.134386,0.142166,0.134696,0.123549,0.100054,0.072651
